In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import japanize_matplotlib
from scipy.linalg import expm
from scipy.stats import norm
from math import factorial
import pandas as pd
import tqdm
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import beta

In [2]:
# %%

# import math # roundは組み込み関数のため不要
# jax関連のライブラリを全てインポート
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax import vmap

# from LikelihoodEvaluator import LikelihoodEvaluator
# from ParameterEstimator import ParameterEstimator
from sympy import Array, symbols

from degenerate_sim import (
    DegenerateDiffusionProcess,
    LikelihoodEvaluator,
    m_estimate,
    newton_solve,
    m_estimate_jax,
    bayes_estimate
)
from degenerate_sim.evaluation.likelihood_evaluator_jax import SymbolicPreparationError

x_sym, y_sym = symbols("x, y")
theta_10, theta_20, theta_21 = symbols("theta_10 theta_20 theta_21")
theta_30, theta_31 = symbols("theta_30 theta_31")

# ベクトル化（1次元テンソルとして）
x = Array([x_sym])
y = Array([y_sym])
theta_1 = Array([theta_10])
theta_2 = Array([theta_20, theta_21])
theta_3 = Array([theta_30, theta_31])

# Step 2: A, B, H の式定義
A = Array([theta_20 * y_sym - x_sym + theta_21])
B = Array([[theta_10]])
H = Array([(y_sym - y_sym**3 - x_sym + theta_31) / theta_30])

# インスタンス
FNmodel = DegenerateDiffusionProcess(
    x=x,
    y=y,
    theta_1=theta_1,
    theta_2=theta_2,
    theta_3=theta_3,
    A=A,
    B=B,
    H=H,
)
FN_likelihood = LikelihoodEvaluator(FNmodel)

true_theta1 = jnp.array([0.3])
true_theta2 = jnp.array([1.5, 0.8])
true_theta3 = jnp.array([0.1, 0.0])
true_theta = (true_theta1, true_theta2, true_theta3)
t_max = 100
burn_out = 50
h = 0.05
bounds_theta1 = [(0.1, 0.5)]
bounds_theta2 = [(0.5, 2.5), (0.5, 1.5)]
bounds_theta3 = [(0.01, 0.3), (-1, 1)]
n = t_max//h

x_data, y_data = FNmodel.simulate(
    true_theta, t_max=t_max, h=h, burn_out=burn_out
)
x_data = np.array(x_data)
y_data = np.array(y_data)
Z = np.array([x_data, y_data]).T

In [3]:
import sympy as sp
import numpy as np

# 変数とパラメータの定義
x, y = sp.symbols('x y')
theta_1, theta_21, theta_22, theta_31, theta_32 = sp.symbols('theta_1 theta_21 theta_22 theta_31 theta_32')

# 演算子Lの定義
def L(f):
    return (theta_21 * y - x + theta_22) * sp.diff(f, x) + \
           (1 / theta_31) * (y - y**3 - x + theta_32) * sp.diff(f, y) + \
           (1 / 2) * theta_1**2 * sp.diff(f, x, x)

# キャッシュを使って演算子を適用する関数
op_cache = {}
def apply_operator(f, k):
    # 式の文字列表現をキャッシュキーに使用する
    f_key = sp.srepr(f)

    # キャッシュがすでに存在する場合、その結果を返す
    if (f_key, k) in op_cache:
        return op_cache[(f_key, k)]

    # 初期状態のキャッシュ保存
    if k == 0:
        op_cache[(f_key, k)] = f
    else:
        # 再帰的に演算子を適用してキャッシュする
        op_cache[(f_key, k)] = L(apply_operator(f, k - 1))

    return op_cache[(f_key, k)]

# 関数をキャッシュして数値計算用に変換する
func_cache = {}
def func(f, k):
    # 式の文字列表現をキャッシュキーに使用する
    f_key = sp.srepr(f)

    # キャッシュが存在する場合、それを返す
    if (f_key, k) in func_cache:
        return func_cache[(f_key, k)]

    # シンボルをリストとして定義
    theta_symbols = (theta_1, theta_21, theta_22, theta_31, theta_32)

    # lambdify を使用して数値関数に変換
    F = apply_operator(f, k)
    numeric_func = sp.lambdify((x, y) + theta_symbols, F)

    # 数値関数をキャッシュ
    func_cache[(f_key, k)] =  lambda x_val, y_val, theta_values: numeric_func(x_val, y_val, *theta_values)/factorial(k)
    return func_cache[(f_key, k)]

# キャッシュをクリアする関数
def clear_cache():
    op_cache.clear()
    func_cache.clear()

# 利用例
# 初期関数 x に演算子を 3 回適用し、その結果を数値計算用に変換
f = x
k = 3

# apply_operator を使用して演算子を適用
f_result = apply_operator(f, k)

# 数値結果を計算
x_val = 1.0
y_val = 2.0
theta_values = [0.3, 1.5, 0.8, 0.1, 0.0]
result = func(f, k)(x_val, y_val, theta_values)
print("Result:", result)



Result: 1935.9666666666665


In [4]:
def update_1(theta_, Z, k0, h):
    x_current = Z[0][1:]
    y_current = Z[1][1:]
    x_previous = Z[0][:-1]
    y_previous = Z[1][:-1]
    theta1 = theta_[0]
    approximated_x = sum(func(x,k)(x_previous, y_previous, theta_)*(h**k) for k in range(k0))
    approximated_xx = sum(func(x*x,k)(x_previous, y_previous, theta_)*(h**k) for k in range(k0 + 1) if k != 1)
    approximated_x_2 = sum(func(x,i)(x_previous, y_previous, theta_)*func(x,j)(x_previous,y_previous,theta_)*(h**(i+j)) for i in range(k0) for j in range(k0) if (i+j <= k0-1)&(i+j != 1))
    theta1_new = np.mean(((x_current -approximated_x)**2 - (approximated_xx - approximated_x_2 ))/h )
    if theta1_new < 0:
        theta1_new = 0
    else:
        theta1_new = np.sqrt(theta1_new)
    return theta1_new

def update_1_final(theta_,Z,k0,h):
    x_current = Z[0][1:]
    y_current = Z[1][1:]
    x_previous = Z[0][:-1]
    y_previous = Z[1][:-1]
    theta_1, theta_21, theta_22, theta_31, theta_32 = theta_
    approximated_x = sum(func(x,k)(x_previous, y_previous, theta_)*(h**k) for k in range(k0))
    approximated_y = sum(func(y,k)(x_previous, y_previous, theta_)*(h**k) for k in range(k0+1))
    approximated_xx = sum(func(x*x,k)(x_previous, y_previous, theta_)*(h**k) for k in range(k0+1))
    approximated_xy = sum(func(x*y,k)(x_previous, y_previous, theta_)*(h**k) for k in range(k0+2))
    approximated_yy = sum(func(y*y,k)(x_previous, y_previous, theta_)*(h**k) for k in range(k0+3))
    approximated_x_2 = sum(func(x,i)(x_previous, y_previous, theta_)*func(x,j)(x_previous,y_previous,theta_)*(h**(i+j)) for i in range(k0+1) for j in range(k0+1) if i+j <= k0)
    approximated_y_2 = sum(func(y,i)(x_previous, y_previous, theta_)*func(y,j)(x_previous,y_previous,theta_)*(h**(i+j)) for i in range(k0+3) for j in range(k0+3) if i+j <= k0+2)
    approximated_xy_2 = sum(func(x,i)(x_previous, y_previous, theta_)*func(y,j)(x_previous,y_previous,theta_)*(h**(i+j)) for i in range(k0+2) for j in range(k0+2) if i+j <= k0+1)
    S0 = np.array([[1,-1/2/theta_31],[-1/2/theta_31,1/3/(theta_31**2)]])
    T0 = np.linalg.inv(np.array([[1,-1/2/theta_31],[-1/2/theta_31,1/3/(theta_31**2)]]))
    theta1_new = np.mean(T0[0,0]*(x_current - approximated_x)**2/h + 2*T0[0,1]*(x_current - approximated_x)*(y_current -approximated_y) / (h**2) + T0[1,1]*(y_current - approximated_y)**2/(h**3))
    theta1_new -= T0[0,0]*(np.mean(approximated_xx -approximated_x_2)/h - theta_1**2*S0[0,0])
    theta1_new -= 2*T0[0,1]*(np.mean(approximated_xy - approximated_xy_2)/(h**2)  - theta_1**2*S0[0,1])
    theta1_new -= T0[1,1]*(np.mean(approximated_yy - approximated_y_2)/(h**3) - theta_1**2 *S0[1,1])
    if theta1_new < 0:
        theta1_new = 0
    return np.sqrt(theta1_new/2)

In [5]:
def update_2(theta_,Z,k0,h):
    x_current = Z[0][1:]
    y_current = Z[1][1:]
    x_previous = Z[0][:-1]
    y_previous = Z[1][:-1]
    approximated_x = sum(func(x,k)(x_previous, y_previous, theta_)*(h**k) for k in range(k0+1) if k != 1) - h*x_previous
    matrix  = np.zeros((2,2))
    matrix[0,0] = -h*np.sum(y_previous**2)
    matrix[0,1] = -h*np.sum(y_previous)
    matrix[1,0] = -h*np.sum(y_previous)
    matrix[1,1] = -h*len(y_previous)
    vector = np.zeros(2)
    vector[0] = -np.sum((x_current - approximated_x)*y_previous)
    vector[1] = -np.sum(x_current - approximated_x)
    theta_new =  np.linalg.inv(matrix) @ vector
    return theta_new
def update_3(theta_,Z,k0,h):
    x_current = Z[0][1:]
    y_current = Z[1][1:]
    x_previous = Z[0][:-1]
    y_previous = Z[1][:-1]
    theta_1, theta_21, theta_22, theta_31, theta_32 = theta_
    approximated_x = sum(func(x,k)(x_previous, y_previous, theta_)*(h**k) for k in range(k0+1))
    approximated_y = sum(func(y,k)(x_previous, y_previous, theta_)*(h**k) for k in range(k0+2) if k != 1)
    Q = y_previous - y_previous**3 - x_previous 
    matrix = np.zeros((2, 2))
    matrix[0, 0] = np.sum(-2*( Q**2) *h)
    matrix[0, 1] = np.sum( -2 *Q*h)
    matrix[1, 0] = np.sum(-2*Q*h)
    matrix[1, 1] = -2* h*len(y_previous)
    
    vector = np.zeros(2)
    vector[0] = np.sum(-2*(y_current-approximated_y)*Q  + h*Q*(x_current-approximated_x)/theta_31)
    vector[1] = np.sum(-2*(y_current-approximated_y) + h*(x_current-approximated_x)/theta_31)

    theta_4 = np.linalg.inv(matrix) @ vector
    theta_31_new = 1/theta_4[0]
    theta_32_new = theta_4[1]/theta_4[0]
    theta_3_new = np.array([theta_31_new,theta_32_new])

    return theta_3_new

In [10]:
# メイン処理
theta_initial = np.array([0.2, 0.5, 0.5, 0.2, 0.1])
theta_true  = np.array([0.3,1.5,0.8,0.1,0.0])
t_max = 100.0
burn_out = 50.0
dt = 0.001
h = 0.05
n = int(t_max // h)
# 複数のseedでループ
seeds = range(50)
k0_range = range(1,6)
theta_results = {k: [] for k in k0_range}
scaled_theta_results = {k: [] for k in k0_range}
for seed in tqdm(seeds):
    x_t,y_t = FNmodel.simulate(true_theta, t_max = 100, h= 0.05, burn_out=50, seed=seed,dt = 0.001)
    Z = np.array([x_t, y_t])
    theta_initial = np.array([0.5,0.5,0.5,0.5,0.5])
    theta = theta_initial.copy()
    scaled_theta = theta_initial.copy()
    for k in k0_range:
        theta = theta_initial.copy()
        theta1 = update_1(theta, Z, k, h)
        theta_initial[0] = theta1
        theta2 = update_2(theta_initial, Z, k, h)
        theta_initial[1] = theta2[0]
        theta_initial[2] = theta2[1]
        theta3 = update_3(theta_initial, Z, k, h)
        theta_initial[3] = theta3[0]
        theta_initial[4] = theta3[1]
        theta_f = theta_initial.copy()
        theta1_f = update_1_final(theta_f, Z, k +1, h)
        theta_f[0] = theta1_f
        if k == 1:
            theta3_f = update_3(theta_f, Z, k, h)
            theta_f[3] = theta3_f[0]
            theta_f[4] = theta3_f[1]
        theta_results[k].append(theta_f.copy())
        scaled_theta[0] = (theta1-theta_true[0]) * np.sqrt(n)
        scaled_theta[1] = (theta2[0]-theta_true[1]) * np.sqrt(n*h)
        scaled_theta[2] = (theta2[1]-theta_true[2]) * np.sqrt(n*h)
        scaled_theta[3] = (theta3[0] -theta_true[3])* np.sqrt(n/h)
        scaled_theta[4] = (theta3[1] -theta_true[4])* np.sqrt(n/h)
        scaled_theta_results[k].append(scaled_theta.copy())


theta_results = {k: np.array(theta_results[k]) for k in k0_range}
scaled_theta_results = {k: np.array(scaled_theta_results[k]) for k in k0_range}

# 統計量の計算
theta_means = {k: np.mean(theta_results[k], axis=0) for k in k0_range}
theta_stds = {k: np.std(theta_results[k], axis=0) for k in k0_range}
theta_stds_0 = {k: np.sqrt(np.mean((theta_results[k]-theta_true)**2, axis=0)) for k in k0_range}

# 結果の表示
results = []
for k in k0_range:
    mean_values = theta_means[k]
    std_values = theta_stds[k]
    std_0_values = theta_stds_0[k]
    
    results.append({
        'k0': k,
        'mean_theta1': mean_values[0],
        'mean_theta21': mean_values[1],
        'mean_theta22': mean_values[2],
        'mean_theta31': mean_values[3],
        'mean_theta32': mean_values[4],
        'std_theta1': std_values[0],
        'std_theta21': std_values[1],
        'std_theta22': std_values[2],
        'std_theta31': std_values[3],
        'std_theta32': std_values[4],
        'std_0_theta1': std_0_values[0],
        'std_0_theta21': std_0_values[1],
        'std_0_theta22': std_0_values[2],
        'std_0_theta31': std_0_values[3],
        'std_0_theta32': std_0_values[4]
    })

results_df = pd.DataFrame(results)

# # 理論的な標準偏差の計算
# theoretical_std_0 = [1/2/np.sqrt(t_max/h), np.sqrt(2)/np.sqrt(t_max), np.sqrt(2)/np.sqrt(t_max), h/np.sqrt(t_max *6)]
# theoretical_std_0_str = np.array2string(np.array(theoretical_std_0), formatter={'float_kind':lambda x: f"{x:.4f}"})
# print("std_theoretical =", theoretical_std_0_str)

results_df

100%|██████████| 50/50 [01:02<00:00,  1.25s/it]


,k0,mean_theta1,mean_theta21,mean_theta22,mean_theta31,mean_theta32,std_theta1,std_theta21,std_theta22,std_theta31,std_theta32,std_0_theta1,std_0_theta21,std_0_theta22,std_0_theta31,std_0_theta32
0,1,0.388968,1.468963,0.782184,0.104335,0.001214,0.010218,0.057471,0.044527,0.000709,0.000503,0.089553,0.065316,0.047959,0.004392,0.001314
1,2,0.304903,1.503007,0.801740,0.100520,0.000896,0.003780,0.059054,0.045677,0.000777,0.000503,0.006191,0.059130,0.045710,0.000935,0.001028
2,3,0.309967,1.511835,0.808792,0.099666,0.000174,0.003819,0.059808,0.046213,0.000851,0.000498,0.010673,0.060968,0.047042,0.000914,0.000528
3,4,0.301231,1.512054,0.809192,0.099881,0.000032,0.003212,0.059846,0.046241,0.000836,0.000491,0.003440,0.061048,0.047146,0.000845,0.000492
4,5,0.302691,1.511952,0.809183,0.099893,0.000008,0.003147,0.059831,0.046234,0.000842,0.000497,0.004141,0.061013,0.047137,0.000848,0.000497


In [11]:
# メイン処理
theta_initial = np.array([0.2, 0.5, 0.5, 0.2, 0.1])
theta_true  = np.array([0.3,1.5,0.8,0.1,0.0])
t_max = 100.0
burn_out = 50.0
dt = 0.001
h = 0.05
n = int(t_max // h)
# 複数のseedでループ
seeds = range(50)
k0_range = range(1,6)
theta_results = {k: [] for k in k0_range}
scaled_theta_results = {k: [] for k in k0_range}
for seed in tqdm(seeds):
    x_t,y_t = FNmodel.simulate(true_theta, t_max = 100, h= 0.05, burn_out=50, seed=seed,dt = 0.01)
    Z = np.array([x_t, y_t])
    theta_initial = np.array([0.5,0.5,0.5,0.5,0.5])
    theta = theta_initial.copy()
    scaled_theta = theta_initial.copy()
    for k in k0_range:
        theta = theta_initial.copy()
        theta1 = update_1(theta, Z, k, h)
        theta_initial[0] = theta1
        theta2 = update_2(theta_initial, Z, k, h)
        theta_initial[1] = theta2[0]
        theta_initial[2] = theta2[1]
        theta3 = update_3(theta_initial, Z, k, h)
        theta_initial[3] = theta3[0]
        theta_initial[4] = theta3[1]
        theta_f = theta_initial.copy()
        theta1_f = update_1_final(theta_f, Z, k +1, h)
        theta_f[0] = theta1_f
        if k == 1:
            theta3_f = update_3(theta_f, Z, k, h)
            theta_f[3] = theta3_f[0]
            theta_f[4] = theta3_f[1]
        theta_results[k].append(theta_f.copy())
        scaled_theta[0] = (theta1-theta_true[0]) * np.sqrt(n)
        scaled_theta[1] = (theta2[0]-theta_true[1]) * np.sqrt(n*h)
        scaled_theta[2] = (theta2[1]-theta_true[2]) * np.sqrt(n*h)
        scaled_theta[3] = (theta3[0] -theta_true[3])* np.sqrt(n/h)
        scaled_theta[4] = (theta3[1] -theta_true[4])* np.sqrt(n/h)
        scaled_theta_results[k].append(scaled_theta.copy())


theta_results = {k: np.array(theta_results[k]) for k in k0_range}
scaled_theta_results = {k: np.array(scaled_theta_results[k]) for k in k0_range}

# 統計量の計算
theta_means = {k: np.mean(theta_results[k], axis=0) for k in k0_range}
theta_stds = {k: np.std(theta_results[k], axis=0) for k in k0_range}
theta_stds_0 = {k: np.sqrt(np.mean((theta_results[k]-theta_true)**2, axis=0)) for k in k0_range}

# 結果の表示
results = []
for k in k0_range:
    mean_values = theta_means[k]
    std_values = theta_stds[k]
    std_0_values = theta_stds_0[k]
    
    results.append({
        'k0': k,
        'mean_theta1': mean_values[0],
        'mean_theta21': mean_values[1],
        'mean_theta22': mean_values[2],
        'mean_theta31': mean_values[3],
        'mean_theta32': mean_values[4],
        'std_theta1': std_values[0],
        'std_theta21': std_values[1],
        'std_theta22': std_values[2],
        'std_theta31': std_values[3],
        'std_theta32': std_values[4],
        'std_0_theta1': std_0_values[0],
        'std_0_theta21': std_0_values[1],
        'std_0_theta22': std_0_values[2],
        'std_0_theta31': std_0_values[3],
        'std_0_theta32': std_0_values[4]
    })

results_df = pd.DataFrame(results)

# # 理論的な標準偏差の計算
# theoretical_std_0 = [1/2/np.sqrt(t_max/h), np.sqrt(2)/np.sqrt(t_max), np.sqrt(2)/np.sqrt(t_max), h/np.sqrt(t_max *6)]
# theoretical_std_0_str = np.array2string(np.array(theoretical_std_0), formatter={'float_kind':lambda x: f"{x:.4f}"})
# print("std_theoretical =", theoretical_std_0_str)

results_df

100%|██████████| 50/50 [00:45<00:00,  1.11it/s]


,k0,mean_theta1,mean_theta21,mean_theta22,mean_theta31,mean_theta32,std_theta1,std_theta21,std_theta22,std_theta31,std_theta32,std_0_theta1,std_0_theta21,std_0_theta22,std_0_theta31,std_0_theta32
0,1,0.386236,1.484767,0.795481,0.103792,0.001716,0.008403,0.050954,0.047670,0.000649,0.000475,0.086645,0.053183,0.047884,0.003847,0.001781
1,2,0.337970,1.521456,0.816884,0.099923,0.001171,0.007325,0.052418,0.049037,0.000657,0.000477,0.038671,0.056639,0.051862,0.000662,0.001265
2,3,0.340263,1.530131,0.823949,0.099024,0.000565,0.006865,0.053073,0.049590,0.000744,0.000464,0.040844,0.061030,0.055070,0.001227,0.000732
3,4,0.326889,1.530501,0.824364,0.099249,0.000432,0.005117,0.053103,0.049623,0.000760,0.000467,0.027371,0.061239,0.055282,0.001068,0.000636
4,5,0.328601,1.530378,0.824344,0.099278,0.000417,0.005298,0.053090,0.049624,0.000760,0.000471,0.029088,0.061167,0.055274,0.001048,0.000629


In [12]:
#results_dfをlatex形式で出力してください
print(results_df.to_latex(index=False, float_format="%.4f"))

\begin{tabular}{rrrrrrrrrrrrrrrr}
\toprule
k0 & mean_theta1 & mean_theta21 & mean_theta22 & mean_theta31 & mean_theta32 & std_theta1 & std_theta21 & std_theta22 & std_theta31 & std_theta32 & std_0_theta1 & std_0_theta21 & std_0_theta22 & std_0_theta31 & std_0_theta32 \\
\midrule
1 & 0.3862 & 1.4848 & 0.7955 & 0.1038 & 0.0017 & 0.0084 & 0.0510 & 0.0477 & 0.0006 & 0.0005 & 0.0866 & 0.0532 & 0.0479 & 0.0038 & 0.0018 \\
2 & 0.3380 & 1.5215 & 0.8169 & 0.0999 & 0.0012 & 0.0073 & 0.0524 & 0.0490 & 0.0007 & 0.0005 & 0.0387 & 0.0566 & 0.0519 & 0.0007 & 0.0013 \\
3 & 0.3403 & 1.5301 & 0.8239 & 0.0990 & 0.0006 & 0.0069 & 0.0531 & 0.0496 & 0.0007 & 0.0005 & 0.0408 & 0.0610 & 0.0551 & 0.0012 & 0.0007 \\
4 & 0.3269 & 1.5305 & 0.8244 & 0.0992 & 0.0004 & 0.0051 & 0.0531 & 0.0496 & 0.0008 & 0.0005 & 0.0274 & 0.0612 & 0.0553 & 0.0011 & 0.0006 \\
5 & 0.3286 & 1.5304 & 0.8243 & 0.0993 & 0.0004 & 0.0053 & 0.0531 & 0.0496 & 0.0008 & 0.0005 & 0.0291 & 0.0612 & 0.0553 & 0.0010 & 0.0006 \\
\bottomrule
\end{tab